In [4]:
"""
批量读取.csv特征集(bicps or tricps)
对每个人的数据进行单独归一化再merge
"""
import pandas as pd
import sys
sys.path.append('..')
from utils.utils_iFEMG_feature import df_save_csv, df_norm, df_merge_colname

In [5]:
"""
获取文件路径下多个文件夹中.csv格式的特征集

1. 文件夹路径
2. 肌肉bi tri
"""
# 文件夹路径 健康人数据
folder_path = r"E:\fuckinggggg\paper_iFEMG_4th\data"
subject_name_list = ['241128s2', '241204s3', '241204s4', '241206s5', '241206s6']


file_df_list = []
for name in subject_name_list:
    file_path = folder_path + '\\' + name + '\\' + "fea_set.csv"
    try:
        file_df_list.append(pd.read_csv(file_path, header = [0], index_col = None))
    except FileNotFoundError:
        print(f"文件 {file_path} 未读取")
    pass

print(f"文件数: {len(file_df_list)}")


文件数: 5


In [6]:
# 生成需要归一化的列名list
muscle_name_lst = ['RF', 'LF', 'SarT', 'BF', 'SemiT', 'VM']
feature_name_lst = ['initial_pressure_ave', 'FMG', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']
col_lst = []
for muscle in muscle_name_lst:
    for fea in feature_name_lst:
        col_lst.append(muscle + fea)

len(col_lst)

54

In [7]:
"""
对每名被试信号特征分别单独归一化

NOTE:
    - 需要修改保存文件的文件名！！
"""

# 归一化某特征
normed_df_list = [] 
err_df_index = 0    # 如果有key error发生报错，报告所在df的index
for df in file_df_list:
    # 对指定的列进行归一化
    try:
        h_df = df_norm(dataframe=df[df['side'] == 'healthy'], col_name=col_lst, method='z_score')
        a_df = df_norm(dataframe=df[df['side'] == 'affected'], col_name=col_lst, method='z_score')
        normed_df = pd.concat([a_df, h_df], axis = 0, ignore_index=True)
        normed_df_list.append(normed_df)
    except KeyError:
        print(f"index {err_df_index} 存在错误")
        pass
    err_df_index = err_df_index + 1
    pass
# 合并dataframe
final_data_df = pd.concat(normed_df_list, axis = 0, ignore_index = True)

In [8]:
final_data_df

,subject,side,motion_type,label,RFinitial_pressure_ave,RFFMG,RFmav,RFrms,RFwave_length,RFzero_crossing,...,SemiTmean_power_freq,VMinitial_pressure_ave,VMFMG,VMmav,VMrms,VMwave_length,VMzero_crossing,VMslope_sign_change,VMmean_freq,VMmean_power_freq
0,241128s2,affected,qian,0.0,0.752485,-0.428352,0.329448,0.681695,1.101179,1.071944,...,-1.131997,-1.273612,-1.006979,-0.744626,-0.588665,-0.726198,-0.523945,2.581633,-0.876736,-0.929566
1,241128s2,affected,qian,0.0,0.752485,-0.899854,1.047155,0.822488,0.681090,-1.635222,...,-1.133412,-1.273612,-1.436965,-0.435984,-0.035511,-0.636153,-0.927388,-0.076157,-0.786561,-0.782346
2,241128s2,affected,qian,0.0,0.752485,-0.174328,-0.005703,0.623954,0.681109,2.268440,...,-0.702750,-1.273612,-0.699362,-0.631736,-0.287867,-0.705165,-0.946102,1.493874,-0.857919,-0.924915
3,241128s2,affected,qian,0.0,0.752485,-0.367778,0.239311,0.706075,0.328863,0.785314,...,-0.650732,-1.273612,-0.434836,-0.669685,-0.474161,-0.701455,-0.321115,1.028299,-0.708094,-0.824413
4,241128s2,affected,qian,0.0,0.752485,-0.196038,0.662971,0.714514,0.649762,-0.746958,...,0.217581,-1.273612,-0.321577,-0.747155,-0.634527,-0.722887,-1.223468,1.013098,-0.809140,-0.765498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,241206s6,healthy,hou,3.0,1.764452,2.022800,-1.200945,-1.430334,-1.198125,1.008592,...,1.337884,0.860597,0.589466,-1.429234,-1.542503,-1.536163,1.327235,1.651277,1.316361,1.046309
302,241206s6,healthy,hou,3.0,1.764452,1.892882,-0.802012,-0.414135,-0.904421,-0.115578,...,1.319188,0.860597,0.937927,-0.529732,-0.354589,-0.480549,0.080572,0.081993,-0.246768,-0.291765
303,241206s6,healthy,hou,3.0,1.764452,1.529304,-0.861533,-0.803897,-0.974582,0.627478,...,1.272679,0.860597,0.889387,-0.443426,-0.363520,-0.483027,-0.364554,-0.394796,-0.248698,-0.307659
304,241206s6,healthy,hou,3.0,1.764452,1.629918,-0.817664,-0.719149,-1.003765,-0.493997,...,1.319564,0.860597,0.733692,-0.467469,-0.360692,-0.444079,-0.000299,-0.022549,-0.286658,-0.231753


In [10]:
# 保存文件
df_save_csv(final_data_df, r"E:\fuckinggggg\paper_iFEMG_4th\data\fea_normed.csv")